In [1]:
# Import modules.

import chaospy as cp
import numpy as np
import pandas as pd
from numpy.testing import assert_array_almost_equal as aaae

from shapley_econsa import _r_condmvn
from shapley_econsa import get_shapley


In [78]:
def test_get_shapley_exact_linear():
    def linear_model(x):
        beta = np.array([[beta_1], [beta_2]])
        return x.dot(beta)

    def x_all(n):
        return cp.MvNormal(mean, cov).sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        if subsetj_conditional is None:
            cov_int = np.array(cov).take(subset_j, axis=1)[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            return distribution.sample(n)
        else:
            return _r_condmvn(
                n,
                mean=mean,
                cov=cov,
                dependent_ind=subset_j,
                given_ind=subsetj_conditional,
                x_given=xjc,
            )

    np.random.seed(123)
    n_inputs = 2
    mean = np.zeros(n_inputs)
    var_1 = 1.5
    var_2 = 2.5
    
    # rho is the correlation coefficient, and thus, in range [-1,1].
    rho = 0.3
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_2)
    beta_1 = 1.3
    beta_2 = 1.5
    beta = (beta_1, beta_2)

    component_1 = beta_1**2 * var_1
    component_2 = beta_2**2 * var_2
    var_y = component_1 + 2 * covariance * beta_1 * beta_2 + component_2
    share = 0.5 * (rho**2)
    true_shapley_1 = (component_1 * (1 - share) + covariance * beta_1 * beta_2 + component_2 * share)/var_y
    true_shapley_2 = (component_2 * (1 - share) + covariance * beta_1 * beta_2 + component_1 * share)/var_y

    cov = np.array(
        [[var_1, covariance],
        [covariance, var_2]]
        )

    method = "exact"
    n_perms = None
    n_output = 10 ** 4
    n_outer = 10 ** 3
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [true_shapley_1, true_shapley_2],
            [0, 0],
            [true_shapley_1, true_shapley_2],
            [true_shapley_1, true_shapley_2],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        linear_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated["Shapley effects"], expected["Shapley effects"])

In [79]:
test_get_shapley_exact_linear()

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 2 / 2 (100%)
Max absolute difference: 0.00085983
Max relative difference: 0.00235477
 x: array([0.364286, 0.635714])
 y: array([0.365146, 0.634854])

## Calculate true Shapley values (as in test function).

In [75]:
    var_1 = 1.5
    var_2 = 2.5
    
    rho = 0.3
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_2)
    beta_1 = 1.3
    beta_2 = 1.5
    beta = (beta_1, beta_2)

    component_1 = beta_1**2 * var_1
    component_2 = beta_2**2 * var_2
    var_y = component_1 + 2 * covariance * beta_1 * beta_2 + component_2
    share = 0.5 * (rho**2)
    true_shapley_1 = (component_1 * (1 - share) + covariance * beta_1 * beta_2 + component_2 * share)/var_y
    true_shapley_2 = (component_2 * (1 - share) + covariance * beta_1 * beta_2 + component_1 * share)/var_y
print(true_shapley_1, '\n', true_shapley_2)

0.3651456842665369 
 0.6348543157334631


## Look for rounding issues.

In [77]:
    var_1 = 1.5
    var_2 = 2.5
    
    rho = 0.3
    #covariance = 
    beta_1 = 1.3
    beta_2 = 1.5
    #beta = (beta_1, beta_2)

    component_1 = (beta_1**2 * var_1)
    component_2 = (beta_2**2 * var_2)
    var_y = ((beta_1**2 * var_1) + 2 * (rho * np.sqrt(var_1) * np.sqrt(var_2)) * beta_1 * beta_2 + (beta_2**2 * var_2))
    share = 0.5 * (rho**2)
    true_shapley_1 = ((beta_1**2 * var_1) * (1 - share) + (rho * np.sqrt(var_1) * np.sqrt(var_2)) * beta_1 * beta_2 + (beta_2**2 * var_2) * share)/((beta_1**2 * var_1) + 2 * (rho * np.sqrt(var_1) * np.sqrt(var_2)) * beta_1 * beta_2 + (beta_2**2 * var_2))
    true_shapley_2 = ((beta_2**2 * var_2) * (1 - share) + (rho * np.sqrt(var_1) * np.sqrt(var_2)) * beta_1 * beta_2 + (beta_1**2 * var_1) * share)/((beta_1**2 * var_1) + 2 * (rho * np.sqrt(var_1) * np.sqrt(var_2)) * beta_1 * beta_2 + (beta_2**2 * var_2))
print(true_shapley_1, '\n', true_shapley_2)

0.3651456842665369 
 0.6348543157334631
